# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook
%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
    "com.snapchat.android_11.4.1.64-2116_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.snapchat.android_11.4.5.73-2117_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
    "com.instagram.android_165.1.0.29.119-253447811_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_166.0.0.41.245-255959626_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_165.1.0.29.119-253447811_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_166.0.0.41.245-255959626_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`
    * GraphGuard extracts the declarations and uses the given names for Auto-Updates.


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

Note that classes of methods and fields are automatically merged with `c_decs`.

In [5]:
named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPEXPERIMENTHELPER': 'X.1fr',
    'CLIPITEM': 'X.9Kh',
    'COMMENT_LONGCLICK_HANDLER': 'X.3S3',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3Jq',
    'DM_HOLDER': 'X.5Et',
    'DM_REPLY': 'X.4Gs',
    'DM_VOICEMESSAGE': 'X.54o',
    'DMVIDEOPLAYER': 'X.5or',
    'FEEDITEM': 'X.1vJ',
    'FRAGMENT': 'androidx.fragment.app.Fragment',
    'HEADERGENERATOR': 'X.0SV',
    'HTTPREQUESTTASKFACTORY': 'X.0s4',
    'INAPP_NOTIFICATION': 'X.Dsu',
    'IGTV_VIEWER_FRAGMENT': 'X.742',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.12J',
    'JACKSON_CREATOR': 'X.0jF',
    'LIVE_POST': 'X.AAL',
    'MAINFEED': 'X.1RI',
    'MEDIAITEM': 'X.1f9',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1nV',
    'NOTIFICATION_HANDLER': 'X.2d8',
    'PHOTOTIMERCONTROLLER': 'X.2yE',
    'POSTDETAIL': 'X.1ei',
    'POSTGESTURE_SIMPLEPOST': 'X.23B',
    'POSTGESTURE_CAROUSELIMAGE': 'X.2CM',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2WC',
    'POSTVIEW': 'X.1en',
    'PROFILE': 'X.0li',
    'PROFILEDETAIL': 'X.3Aa',
    'PROFILEPIC_LONGCLICK': 'X.3Ah',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2MK',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0o2',
    'RESPONSE_ASYNC_HANDLER': 'X.1Jk',
    'NOTIFICATION_RUNNABLE': 'X.Dsv',
    'SERIALSCHEDULER': 'X.0j6',
    'SESSION': 'X.0Nr',
    'SIMPLE_EXECUTOR': 'X.4Oe',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1ow',
    'VIDEOSETTINGSMANAGER': 'X.1ha',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPEXPERIMENTHELPER_ISREDESIGN': MethodDec('X.1fr', 'A08', 'X.0Nr'),
    'CLIPITEM_UPDATEMEDIA': MethodDec('X.9Kh', 'A03', 'X.9Kh', 'X.9Kq', 'X.2BH', 'int', 'java.lang.String'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3S3', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Gv', 'androidx.fragment.app.FragmentActivity', 'X.0Nr', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Nr'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Nr'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3Jq', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.5Et', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.4Gs', 'A0K', 'X.4Gs', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.4Gs', 'A0H', 'X.4Gs'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.5or', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1vJ', 'A00', 'X.0jX'),
    'FRAGMENT_GETACTIVITY': MethodDec('androidx.fragment.app.Fragment', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0SV', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0s4', 'A03', 'java.lang.String', 'X.0Nr'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.Dsu', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.742', 'A00', 'X.742'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.742', 'A0C', 'X.742'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.12J', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0jF', 'A03', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0jF', '<init>', 'X.Dkg'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.AAL', 'A00', 'X.0kO', 'X.AAM'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'ALv'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1RI', 'configureActionBar', 'X.1LY'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1f9', 'A01', 'X.0jX', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0kO', 'X.1f9'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1nV', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2d8', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2yE', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1ei', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.23B', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.2CM', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2WC', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1en', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3Aa', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0li', 'A04', 'X.0Nr', 'X.0kA'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3Ah', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2MK', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BQt', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0e', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0M', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0o', 'X.1pt', 'X.1vp'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0o2', 'A00', 'X.0o1', 'X.1Jg'),
    'RESPONSE_ASYNC_HANDLER_ONRESPONSE': MethodDec('X.1Jk', 'A02', 'X.1Jg', 'X.2A7'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.Dsv', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0j6', 'A02', 'X.0q9'),
    'SESSION_GETTOKEN': MethodDec('X.0Nr', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4Oe', 'onFail', 'X.1vL'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4Oe', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4Oe', '<init>', 'X.0Nr'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1ow', 'A03', 'X.0kO', 'X.0k9'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1ha', 'A00', 'X.0Nr'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1LX', 'A09'),
    'CLIPSITEM_SESSION': FieldDec('X.9Kh', 'A03'),
    'CLIPSVIEW_PARENTVIEW': FieldDec('X.9Kq', 'A00'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.2BH', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3S3', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1pW', 'A0a'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3Jq', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.5FB', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.5FE', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.54o', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1vJ', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0za', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.742', 'A0Y'),
    'MAINFEED_SESSION': FieldDec('X.1RI', 'A0a'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1vF', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1vp', 'A0A'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1vp', 'A09'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.1Ky', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1n4', 'A01'),
    'NATIVEINPUTSTREAMASYNCHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1Jl', 'A03'),
    'NATIVEINPUTSTREAMASYNCHOLDER_URI': FieldDec('X.1Jl', 'A04'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.Dsu', 'A00'), FieldDec('X.Dsv', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.Dsu', 'A01'), FieldDec('X.Dsv', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1tY', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1tb', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3Ad', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3Ad', 'A05'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3Ad', 'A06'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3Ad', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3Af', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.1Kw', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2yM', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1I'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1n3', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1vp', 'A0E')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:

# c_decs = tuple()
# m_decs = tuple()
# f_decs = tuple()

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1nV#A01() -> X.1or#A01()
+ Found single candidate for Method. Considering it a match 
	X.0SV#A00(android.content.Context) -> X.0SL#A00(android.content.Context)
* Found multiple candidates for Method X.1fr#A08(X.0Nr)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0o(X.1pt, X.1vp) -> com.instagram.reels.fragment.ReelViewerFragment#A0j(X.1qy, X.1ww)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidat

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

~ Unique String not used in single class anymore. Change! (unavailable)
~ Unique String not used in single class anymore. Change! (is_viewer_comment_allowed)
.. Considering ambiguous match by UniqueString as match with a majority of 0.99 (LX/1RI; -> LX/1SX;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Nr; -> LX/0Nv;
+ Found single candidate for Class. Considering it a match! 
	LX/0j6; -> LX/0ju;
+ Found single candidate for Class. Considering it a match! 
	LX/0s4; -> LX/0rQ;
+ Found single candidate for Class. Considering it a match! 
	LX/0za; -> LX/0ru;
+ Found single candidate for Class. Considering it a match! 
	LX/12J; -> LX/13o;
+ Found single candidate for Class. Considering it a match! 
	LX/1Jl; -> LX/0tc;
+ Found single candidate for Class. Considering it a match! 
	LX/1Kw; -> LX/0vC;
+ Found single candidate for Class. Considering it a match! 
	LX/1Ky; -> LX/2CT;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigati

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/3S3; -> LX/3Ug;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3Jq; -> LX/3Mv;
+ Found single candidate for Class. Considering it a match! 
	LX/5Et; -> LX/5KL;
+ Found single candidate for Class. Considering it a match! 
	LX/5or; -> LX/5Wr;
+ Found single candidate for Class. Considering it a match! 
	LX/0jF; -> LX/0k3;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2yE; -> LX/34I;
+ Found single candidate for Class. Considering it a match! 
	LX/23B; -> LX/2EM;
* Found multiple candidates for Class LX/2CM;
* Found multiple candidates for Class LX/2W

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.2CM#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2WC#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.1en#A01(X.1M2, X.1MA, X.1f9, X.20Q, int, X.1tb, java.lang.String)
* Found multiple candidates for Method X.1fr#A08(X.0Nr)
- Inner join on possible candidates resulted in no match for Method X.1fr#A08(X.0Nr)
+ Found single candidate for Method. Considering it a match 
	X.9Kh#A03(X.9Kh, X.9Kq, X.2BH, int, java.lang.String) -> X.9o0#A03(X.9o0, X.9oV, X.206, int, java.lang.String)
+ Found single candidate for Method. Considering it a match 
	X.3S3#onLongPress(android.view.MotionEvent) -> X.3Ug#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1Gv, androidx.fragment.app.FragmentActivity, X.0Nr, android.os.Bundle) -> com.instagram.debu

+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0e(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0a(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String)
* Found multiple candidates for Method com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment)
+ Found single candidate for Method. Considering it a match 
	X.0j6#A02(X.0q9) -> X.0ju#A02(X.0pJ)
+ Found single candidate for Method. Considering it a match 
	X.0Nr#getToken() -> X.0Nv#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1ow#A03(X.0kO, X.0k9) -> X.1YQ#A03(X.0lA, X.0kv)
Could resolve 0 new Classes, 15 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0k9; -> LX/0kv;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1vP; -> LX/1a5;
+ Found single candidate for Class. Considering it a match! 
	LX/AAM; -> LX/1ue;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/14b; -> LX/0ry;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/gesture/GestureManagerFrameLayout; -> Lcom/instagram/ui/gesture/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a 

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

76 / 153 Classes were matched
54 / 62 Methods were matched
38 / 43 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1nV -> X.1or
• X.0SV -> X.0SL
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.4Gs -> X.44g
• X.0o2 -> X.0oV
• X.1ha -> X.1iS
• X.1Jk -> X.2CP
• X.2MK -> X.2JX
• X.4Oe -> X.4Dn
• X.Dsu -> X.6mp
• X.Dsv -> X.6mq
• X.0Nr -> X.0Nv
• X.0j6 -> X.0ju
• X.0s4 -> X.0rQ
• X.0za -> X.0ru
• X.12J -> X.13o
• X.1Jl -> X.0tc
• X.1Kw -> X.0vC
• X.1Ky -> X.2CT
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1LX -> X.1N5
• androidx.fragment.app.

• androidx.fragment.app.Fragment#getActivity() -> androidx.fragment.app.Fragment#getActivity()
• X.0s4#A03(java.lang.String, X.0Nr) -> X.0rQ#A03(java.lang.String, X.0Nv)
• X.742#A00(X.742) -> X.7DM#A00(X.7DM)
• X.AAL#A00(X.0kO, X.AAM) -> X.1ud#A00(X.0lA, X.1ue)
• X.1f9#A01(X.0jX, boolean) -> X.1Xl#A00(X.0kL, boolean)
• com.instagram.feed.media.Media__JsonHelper#A00(X.0kO, X.1f9) -> com.instagram.feed.media.Media__JsonHelper#A00(X.0lA, X.1Xl)
• X.2d8#A01(android.content.Context, java.lang.CharSequence, int) -> X.5WL#A01(android.content.Context, java.lang.CharSequence, int)
• X.1ei#A05(X.1tY, X.1f9, X.20Q, int, boolean, java.lang.String, X.0Nr, X.1Qq, java.lang.Integer, X.1i6, X.1RW) -> X.1fm#A06(X.1ux, X.1Xl, X.1xZ, int, boolean, java.lang.String, X.0Nv, X.1S5, java.lang.Integer, X.1j2, X.1Sn)
• com.instagram.reels.fragment.ReelViewerFragment#A0e(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0a(com.instagram.reels.

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    # No match for Method X.1fr#A08(X.0Nr),
    MethodDec('X.9o0', 'A03', 'X.9o0', 'X.9oV', 'X.206', 'int', 'java.lang.String'),
    MethodDec('X.3Ug', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1JJ', 'androidx.fragment.app.FragmentActivity', 'X.0Nv', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nr),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nr),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.

    'X.4q6'
    'X.5Wr'
    'X.1XZ'
    'androidx.fragment.app.Fragment'
    'X.0SL'
    'X.0rQ'
    'X.6mp'
    'X.7DM'
    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.13o'
    'X.0k3'
    'X.1ud'
    'X.1SX'
    'X.1Xl'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1or'
    'X.5WL'
    'X.34I'
    'X.1fm'
    'X.2EM'
    # No Match Found for Class 'X.2CM'
    # No Match Found for Class 'X.2WC'
    # No Match Found for Class 'X.1en'
    'X.0mO'
    'X.3Aw'
    'X.3B5'
    'com.facebook.proxygen.JniHandler'
    'X.2JX'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.0oV'
    'X.2CP'
    'X.6mq'
    'X.0ju'
    'X.0Nv'
    'X.4Dn'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1YQ'
    'X.1iS'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1N5', 'A09'),
    FieldDec('X.9o0', 'A03'),
    # No Match Found for Field FieldDec('X.9Kq', 'A00'),
    FieldDec('X.206', 'A00'),
    FieldDec('X.3Ug',

# Warnings

Shows some warnings that result in common bugs, so that the Hooks (or other kind of modifications applied) must be updated logic-wise. 

Currently supports: 
* Change in number of parameters 

In [20]:
for m_old, m_new in accumulator.matching_ms.items():
    def get_number_of_params(m):
        return len(tuple(get_pretty_params(str(m.get_descriptor()))))
    
    if get_number_of_params(m_old) != get_number_of_params(m_new):
        print("Warning: Number of parameters changed for:", pretty_format_ma(m_old), "-", pretty_format_ma(m_new))

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [21]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
%autoreload
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for X.1fr#A08(X.0Nr)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nr)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Nr)
No matching Method found for X.1RI#configureActionBar(X.1LY)
No matching Method found for X.2CM#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2WC#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.1en#A01(skip.params)
No matching Method found for com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment)
No matching Field found for X.9Kq#A00
No matching Field found for com.instagram.direct.visual.DirectVisualMessageViewerController#mRootView
Only updating matched Class: com.in

In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPEXPERIMENTHELPER = ClassDec("X.2Bw")

    override val CLIPITEM = ClassDec("X.9o0")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3Ug")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3Mv")

    override val DM_HOLDER = ClassDec("X.5KL")

    override val DM_REPLY = ClassDec("X.44g")

    override val DM_VOICEMESSAGE = ClassDec("X.4q6")

    override val DMVIDEOPLAYER = ClassDec("X.5Wr")

    override val FEEDITEM = ClassDec("X.1XZ")

  

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream
import java.net.URI

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1N5"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.9o0"])
    override val CLIPSITEM_SESSION = VariableDec<Any>("A03")

    @FieldClass([/* TODO */ "X.9Kq"])
    override val CLIPSVIEW_PARENTVIEW = /* TODO */ VariableDec<FrameLayout>("A00")

    @FieldClass(["X.206"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3Ug"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1Z9"])
    override val 